Faça o upload de um arquivo .zip com os seus dados e use o comando abaixo após a finalização do upload para a extração do conteúdo do arquivo no colab.

Implemente as funções a seguir.

In [43]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms
import random

import os
import pandas as pd
from PIL import Image
import cv2 as cv
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset

In [58]:
class CarSimDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0].strip())
        image = read_image(img_path)
        
        # Apply color filter
        image = self.apply_color_filter(image)

        if self.transform:
            image = self.transform(image)

        label = self.img_labels.iloc[idx, 1]
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label

    def apply_color_filter(self, image):
        # Convert image to float for calculations
        image = image.float()
        
        # Convert RGB to HSV
        max_rgb, _ = torch.max(image, dim=0, keepdim=True)
        min_rgb, _ = torch.min(image, dim=0, keepdim=True)
        delta = max_rgb - min_rgb
        
        # Hue calculation
        hue = torch.zeros_like(max_rgb)
        mask = max_rgb == image[0]
        hue[mask] = ((image[1] - image[2]) / delta)[mask]
        mask = max_rgb == image[1]
        hue[mask] = 2.0 + ((image[2] - image[0]) / delta)[mask]
        mask = max_rgb == image[2]
        hue[mask] = 4.0 + ((image[0] - image[1]) / delta)[mask]
        hue = (hue / 6.0) % 1.0
        hue[delta == 0] = 0  # To handle division by zero
        
        # Saturation calculation
        saturation = delta / (max_rgb + 1e-6)
        
        # Value calculation
        value = max_rgb / 255.0
        
        hsv = torch.stack([hue, saturation, value], dim=0)

        # Define the range for gray color in HSV
        lower_gray = torch.tensor([0.0, 0.0, 50 / 255], device=image.device)
        upper_gray = torch.tensor([1.0, 50 / 255, 200 / 255], device=image.device)

        # Create mask for gray color
        mask = (hsv >= lower_gray.unsqueeze(1).unsqueeze(2)) & (hsv <= upper_gray.unsqueeze(1).unsqueeze(2))
        mask = mask.all(dim=0)

        # Apply the mask to the original image
        filtered_image = image * mask.float()
        
        return filtered_image



In [59]:
# Transformações de imagem
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.RandomRotation(degrees=5),
    transforms.ColorJitter(brightness=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=[-0.034986, -0.034986, -0.034986], std=[0.073005, 0.073005, 0.073005])
])

# Transformação dos rótulos
def normalize_labels(label):
    max_angle = 25
    return label / max_angle

# Dataset de treino com transformações atualizadas
train_dataset = CarSimDataset(
    annotations_file='C:\\Users\\moura\\Searches\\behavior-cloning\\tesst.csv',
    img_dir='DataAndLoader\\IMG',
    transform=transform,
    target_transform=normalize_labels
)

# Dataset de teste
test_dataset = CarSimDataset(
    annotations_file='C:\\Users\\moura\\Searches\\behavior-cloning\\tesst.csv',
    img_dir='DataAndLoader\\IMG',
    transform=transform,
    target_transform=normalize_labels
)


Crie as variáveis a seguir a partir da classe implementada acima.

O código abaixo será utilizado para treinar o modelo com o seu dataset.

In [60]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [61]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 28, 28])
Shape of y: torch.Size([64]) torch.float32


In [62]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits.squeeze()

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2352, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [63]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [64]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [65]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    # correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [66]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.099058  [   64/22265]
loss:     nan  [ 6464/22265]
loss:     nan  [12864/22265]
loss:     nan  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss:      nan 

Epoch 2
-------------------------------
loss:     nan  [   64/22265]
loss:     nan  [ 6464/22265]
loss:     nan  [12864/22265]
loss:     nan  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss:      nan 

Epoch 3
-------------------------------
loss:     nan  [   64/22265]
loss:     nan  [ 6464/22265]
loss:     nan  [12864/22265]
loss:     nan  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss:      nan 

Epoch 4
-------------------------------
loss:     nan  [   64/22265]
loss:     nan  [ 6464/22265]
loss:     nan  [12864/22265]
loss:     nan  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss:      nan 

Epoch 5
-------------------------------
loss:     nan  [   64/22265]
loss:     nan  [ 6464/22265]
loss:     nan  [12864/22265]
loss:     nan  [19264/22265]
Test Error: 
 A

In [67]:
torch.save(model.state_dict(), "model_caralhooo.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Faça o download do seu modelo após o treinamento, caso queira testá-lo no simulador.

O código a seguir demonstra como o modelo será usado para inferência no simulador. Caso seja necessário, altere a função *preprocess*.

In [ ]:
def preprocess(x):
    # TODO: se necessário, alterar função
    return x

In [ ]:
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

In [ ]:
# model.eval()
# x = test_data[0][0]
# with torch.no_grad():
    # x = preprocess(x)
    # x = x.to(device)
    # pred = model(x)